# Importing important Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import lxml
from threading import Thread, Lock
import mysql.connector
import os 
import shutil
from concurrent.futures import ThreadPoolExecutor

# Declaring important Global variable

In [2]:
url="https://www.stfrancismedicalcenter.com/find-a-provider/? fbclid=IWAR0Gwi2H4D1uqJTOQnCsPcbGhf8EH8_B7z-ZdbsPr8xsMN-9TbY4E-EoPc4"  
formdata={
        "_m_": "FindAPhysician",
        "PhysicianSearch$HDR0$PhysicianName": "",
        "PhysicianSearch$HDR0$SpecialtyIDs": "",
        "PhysicianSearch$HDR0$Distance": "5",
        "PhysicianSearch$HDR0$ZipCodeSearch": "",
        "PhysicianSearch$HDR0$Keywords": "",
        "PhysicianSearch$HDR0$LanguageIDs": "",
        "PhysicianSearch$HDR0$Gender": "",
        "PhysicianSearch$HDR0$InsuranceIDs": "",
        "PhysicianSearch$HDR0$AffiliationIDs": "",
        "PhysicianSearch$HDR0$NewPatientsOnly": "",
        "PhysicianSearch$HDR0$InNetwork": "",
        "PhysicianSearch$HDR0$HasPhoto": "",
        "PhysicianSearch$FTR01$PagingID": "3"
}
name=[]
link=[]
MyDb=mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="allproviders"
)
MyCursor=MyDb.cursor() 
MyCursor.execute("DROP TABLE IF EXISTS ALLPROVIDER")
sql = """
CREATE TABLE IF NOT EXISTS ALLPROVIDER
    (
        id int NOT NULL PRIMARY KEY AUTO_INCREMENT, 
        html text, 
        Images text
);"""
MyCursor.execute(sql)
MyDb.commit()
lock = Lock()
if not os.path.isdir("images"): os.makedirs("images")
if not os.path.isdir("html"): os.makedirs("html")

totalRecord=0

# Creating function for storing data

In [3]:
def storeData(*urlTuple):
    """
        Function for storing data
        It will take a tuple as a parameter
            urlTuple=(htmlFilePath, imgGilePath)
        Return Value:=> None
    """
    global totalRecord
    htmlFilePath,imgFilePath=urlTuple
    #print(htmlFilePath,"StoreData")
    lock.acquire()
    sql="INSERT INTO ALLPROVIDER (html,Images) VALUES (%s,%s)"
    MyCursor.execute(sql,(htmlFilePath, imgFilePath))
    MyDb.commit()
    totalRecord+=1
    print(totalRecord,"Saved Successfully")
    lock.release()

# Function for Geting Profile Images & Html

In [4]:
def get_Images_and_Html(urlTuple):
    """
        This method is used for geting Image and Html data from a url & store it into system.
        It takes tuple which contains two values "Profile Urls" & "Image Url"
            urlTuple=(urls,imgUrl)
        Return Value:=> None
    """
    urls,imgUrl=urlTuple
    url=f"https://www.stfrancismedicalcenter.com{urls}"
    try:
        fileName=urls.split("/")[-2].replace("-","_")
    except:
        fileName=totalRecord
        
    r=requests.get(url)
    bs=BeautifulSoup(r.text,'lxml')
    with open(f"html/{fileName}.html", "wb") as f:
        f.write(bs.encode())
    htmlFilePath=f"html/{fileName}.html"
    
    imgUrl=f"https://www.stfrancismedicalcenter.com{imgUrl}"
    img = requests.get(imgUrl, stream = True)
    if img.status_code == 200:
        with open(f"images/{fileName}.jpg", 'wb') as f: 
            img.raw.decode_content = True
            shutil.copyfileobj(img.raw, f)
            imgfilePath=f"images/{fileName}.jpg"    

    else: 
        imgFilePath=""
    storeData(htmlFilePath,imgfilePath)


# Function for get all pages

In [5]:
def getPages(physician):
    """
        Extracting all pages using formdata/payroll 
                        &
        Getting all Profile Photos and Urls to extract record.
        It takes Page Id as a parameter. 
    """
    global formdata # Use global fordata for changing physician Paging Id for different pages.
    formdata["PhysicianSearch$FTR01$PagingID"]=str(physician) # Changing value of formdata
    r=requests.post(url,data=formdata) # Post request for geting the page
    bs=BeautifulSoup(r.text,"lxml")
    # Use multithreading
    with ThreadPoolExecutor() as executor:
        executor.map(
            get_Images_and_Html,
            [
                (
                    i['href'],
                    i.figure.picture.img["data-src"]
                ) 
                for i in bs.find_all('a',class_="flex-top-between-block-500")
            ]
    )

# Main Function

In [6]:
if __name__=="__main__":
    lst=[i for i in range(1,39)]
    with ThreadPoolExecutor() as e:
        e.map(getPages,lst)
    print("Total Records",totalRecord)

1 Saved Successfully
2 Saved Successfully
3 Saved Successfully
4 Saved Successfully
5 Saved Successfully
6 Saved Successfully
7 Saved Successfully
8 Saved Successfully
9 Saved Successfully
10 Saved Successfully
11 Saved Successfully
12 Saved Successfully
13 Saved Successfully
14 Saved Successfully
15 Saved Successfully
16 Saved Successfully
17 Saved Successfully
18 Saved Successfully
19 Saved Successfully
20 Saved Successfully
21 Saved Successfully
22 Saved Successfully
23 Saved Successfully
24 Saved Successfully
25 Saved Successfully
26 Saved Successfully
27 Saved Successfully
28 Saved Successfully
29 Saved Successfully
30 Saved Successfully
31 Saved Successfully
32 Saved Successfully
33 Saved Successfully
34 Saved Successfully
35 Saved Successfully
36 Saved Successfully
37 Saved Successfully
38 Saved Successfully
39 Saved Successfully
40 Saved Successfully
41 Saved Successfully
42 Saved Successfully
43 Saved Successfully
44 Saved Successfully
45 Saved Successfully
46 Saved Successful

363 Saved Successfully
364 Saved Successfully
365 Saved Successfully
366 Saved Successfully
367 Saved Successfully
368 Saved Successfully
369 Saved Successfully
370 Saved Successfully
371 Saved Successfully
372 Saved Successfully
373 Saved Successfully
374 Saved Successfully
375 Saved Successfully
376 Saved Successfully
377 Saved Successfully
378 Saved Successfully
379 Saved Successfully
380 Saved Successfully
381 Saved Successfully
382 Saved Successfully
383 Saved Successfully
384 Saved Successfully
385 Saved Successfully
386 Saved Successfully
387 Saved Successfully
388 Saved Successfully
389 Saved Successfully
390 Saved Successfully
391 Saved Successfully
392 Saved Successfully
393 Saved Successfully
394 Saved Successfully
395 Saved Successfully
396 Saved Successfully
397 Saved Successfully
398 Saved Successfully
399 Saved Successfully
400 Saved Successfully
401 Saved Successfully
402 Saved Successfully
403 Saved Successfully
404 Saved Successfully
405 Saved Successfully
406 Saved S